# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [3]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [5]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Creation**: Generative AI can generate high-quality content such as:
	* Articles and blog posts
	* Social media posts and captions
	* Product descriptions and e-commerce content
	* Music and audio tracks
2. **Product Design and Development**: Generative AI can assist in product design, prototyping, and development by:
	* Generating 3D models and CAD designs
	* Creating prototypes and mockups
	* Optimizing product performance and functionality
3. **Marketing and Advertising**: Generative AI can help with marketing and advertising efforts by:
	* Generating personalized ad copy and landing pages
	* Creating social media ads and promotions
	* Analyzing customer behavior and sentiment to inform marketing strategies
4. **Data Analysis and Visualization**: Generative AI can aid in data analysis and visualization by:
	* Generating insights and recommendations from large datasets
	*

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

Generative AI has numerous business applications across various industries. Here are some examples:

1. **Content Generation**: Generative AI can be used to generate high-quality content such as articles, social media posts, product descriptions, and even entire books. This can help businesses save time and resources while maintaining a consistent tone and style.
2. **Product Design**: Generative AI can aid in the design of products such as 3D models, logos, and packaging designs. It can also be used to generate new product ideas based on existing product lines.
3. **Marketing Campaigns**: Generative AI can help create personalized marketing campaigns by generating targeted content, social media posts, and even entire ad campaigns.
4. **Image and Video Generation**: Generative AI can be used to generate high-quality images and videos for businesses such as e-commerce websites, social media platforms, and advertising agencies.
5. **Customer Service Chatbots**: Generative AI-powered chat

## Alternative approach - using OpenAI python library to connect to Ollama

In [7]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

Generative AI has numerous business applications across various industries, including:

1. **Content Creation**: Generative AI can generate high-quality content such as text, images, and videos, reducing the need for manual labor and increasing efficiency.
2. **Marketing and Advertising**: Generative AI can create personalized marketing campaigns, product recommendations, and ad copy, improving customer engagement and conversion rates.
3. **Product Design**: Generative AI can design products, prototypes, and even entire product lines, streamlining the innovation process and reducing development costs.
4. **Virtual Productization**: Generative AI can generate digital twins of physical products, allowing for testing, iteration, and virtual marketing without the need for physical prototyping.
5. **Chatbots and Customer Service**: Generative AI-powered chatbots can provide personalized customer support, answer frequently asked questions, and handle routine tasks, freeing up human customer 

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [10]:
class Website:
    
    def __init__(self, url):
        
        self.url = url
        HEADERS = {"Content-Type": "application/json"}
        response = requests.get(self.url, headers = HEADERS)
        soup  = BeautifulSoup(response.content, "html.parser")
        
        self.title = soup.title.string if soup.title else "No Title Found!"
        
        for item in soup.body(["img","script","style","input"]):
            item.decompose() 
        
        self.text = soup.body.get_text(separator="\n", strip=True)

In [11]:
kanit = Website("https://kanitvural.com")
print(kanit.title)
print(kanit.text)

Kanit Vural | Data Scientist & ML Engineer
Home
About
Skills
Projects
Blog
Contact
Kanıt Vural
|
Transforming data into actionable insights and building intelligent solutions
Get in Touch
About Me
Data Scientist & Mine Research & Development Engineer with expertise in AI-driven solutions
He began his career in 2008 at Erdemir Mining Company as a Mining R&D Engineer, part of Oyak Mining Metallurgy Group in Divriği, Turkey. Early on, he worked on geophysical gravity and magnetic iron ore exploration and learned Surpac software to create 3D solid models from drill data. He enhanced his skills in Geostatistics through training at Hacettepe University.
With this expertise, he created block models and conducted reserve classifications, improving the company’s cost-efficiency and profits. He played a key role in discovering new fields and developing existing reserves, contributing to significant financial gains.
In 2020, he joined Tosyalı Iron Steel Angola, a subsidiary of Tosyalı Holding, in

In [12]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [13]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [14]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:

MODEL = "llama3.2"
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [15]:
def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model=MODEL,
        messages=messages_for(website)
    )
    return response.choices[0].message.content

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [16]:
display_summary("https://kanitvural.com")

### Summary of Kanit Vural's Website

Kanit Vural is a Data Scientist & ML Engineer with expertise in AI-driven solutions. His website provides an overview of his skills, experience, and projects.

#### Key Insights

* **15+ years of experience**: Starting as a Mining R&D Engineer in 2008.
* **Strong foundation in geostatistics** through Hacettepe University training.
* **Wide range of technologies**: Python, NumPy, Pandas, Scikit-learn, TensorFlow, PyTorch, and more.
* **Variety of projects**:
	+ Face recognition application
	+ Voice-to-image converter using AI
	+ Chatbot for YouTube videos
	+ Intelligent data analysis with LLM agents
	+ Comprehensive exploration of sentiment analysis and NLP techniques.

#### Featured Projects

1. Smile-Based Face Recognition Access Control System
2. Data Analyzer with LLM Agents
3. Evolution of Sentiment Analysis
4. Fish Species Classification with ANN
5. Cardiovascular Disease Prediction
6. Vegetable Image Classification
7. CRM Analytics & Customer Segmentation
8. Amazon Multi-Model Analysis